In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = '54DYXJOBER2F4X1GZYLKUN1BK5CN5TWVURNKYSAQ052D2M3V' # your Foursquare ID
CLIENT_SECRET = 'IBWPSNOADLPZXKWVHUGENR5DXH5E0I1BI2TGSXIVXY3OSI4L' # your Foursquare Secret
ACCESS_TOKEN = 'WG1DFCNLOYXHROW4EBKPT2SJZIQU1V21OP0O0YJ3QWIVIWUL' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54DYXJOBER2F4X1GZYLKUN1BK5CN5TWVURNKYSAQ052D2M3V
CLIENT_SECRET:IBWPSNOADLPZXKWVHUGENR5DXH5E0I1BI2TGSXIVXY3OSI4L


In [4]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Los Angeles, CA', 'San Francisco, CA', 'Dallas, TX', 'Seattle, WA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d196941735") # Hospital ID
    results[city] = requests.get(url).json()

In [5]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of hospital in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of hospital in New York, NY =  266
Showing Top 100
Total number of hospital in Los Angeles, CA =  210
Showing Top 100
Total number of hospital in San Francisco, CA =  186
Showing Top 100
Total number of hospital in Dallas, TX =  198
Showing Top 100
Total number of hospital in Seattle, WA =  185
Showing Top 100


In [7]:
maps[cities[0]]

In [8]:
maps[cities[1]]

In [9]:
maps[cities[2]]

In [10]:
maps[cities[3]]

In [11]:
maps[cities[4]]

Among the four cities, New York and LA have more hospitals than other two.


In [18]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.023630874515147247
Los Angeles, CA
Mean Distance from Mean coordinates
0.10869656135179911
San Francisco, CA
Mean Distance from Mean coordinates
0.026154080491832876
Dallas, TX
Mean Distance from Mean coordinates
0.10504972567093532
Seattle, WA
Mean Distance from Mean coordinates
0.03792057043214949


In [19]:
maps[cities[0]]

In [20]:
maps[cities[1]]

In [21]:
maps[cities[2]]

In [22]:
maps[cities[3]]

In [23]:
maps[cities[4]]